In [2]:
import pandas as pd
import kaggle
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta
import sys
import os

from darts import TimeSeries
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, InvertibleMapper, StaticCovariatesTransformer
from darts.dataprocessing.transformers.missing_values_filler import MissingValuesFiller
from darts.metrics import rmsle
from darts.models import LinearRegressionModel, LightGBMModel, XGBModel, CatBoostModel
from darts.models.filtering.moving_average_filter import MovingAverageFilter
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from tqdm.notebook import tqdm_notebook

plt.style.use("ggplot")
plt.rcParams["font.size"] = 15
COLORS = list(sns.color_palette())

c:\Users\georg\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [5]:
directory = "~/store-sales-time-series-forecasting/"


holiday_events = pd.read_csv(directory + 'holidays_events.csv', parse_dates=["date"])
oil = pd.read_csv(directory + 'oil.csv',parse_dates=["date"])
sample_submission = pd.read_csv(directory + 'sample_submission.csv')
stores = pd.read_csv(directory + 'stores.csv')
test = pd.read_csv(directory + 'test.csv',parse_dates=["date"])
train = pd.read_csv(directory + 'train.csv', parse_dates=["date"])
transactions = pd.read_csv(directory + 'transactions.csv',parse_dates=["date"])

In [45]:
num_family = train.family.nunique()
num_store = train.store_nbr.nunique()
num_ts = train.groupby(["store_nbr", "family"]).ngroups
train_start = train.date.min().date()
train_end = train.date.max().date()
num_train_date = train.date.nunique()
train_len = (train_end - train_start).days + 1
test_start = test.date.min().date()
test_end = test.date.max().date()
num_test_date = test.date.nunique()
test_len = (test_end - test_start).days + 1

In [46]:
# reindex training data
multi_idx = pd.MultiIndex.from_product(
    [pd.date_range(train_start, train_end), train.store_nbr.unique(), train.family.unique()],
    names=["date", "store_nbr", "family"],
)
train = train.set_index(["date", "store_nbr", "family"]).reindex(multi_idx).reset_index()

# fill missing values with 0s
train[["sales", "onpromotion"]] = train[["sales", "onpromotion"]].fillna(0.)
train.id = train.id.interpolate(method="linear") # interpolate linearly as a filler for the 'id'

In [47]:
missing_dates = pd.date_range(train_start, train_end).difference(train.date.unique())
missing_dates = missing_dates.strftime("%Y-%m-%d").tolist()

unique_dp_count = train.groupby(["store_nbr", "family"]).date.count().unique().tolist()

In [48]:


missing_oil_dates = pd.date_range(train_start, test_end).difference(oil.date)
num_missing_oil_dates = len(missing_oil_dates)
num_wknd_missing = (missing_oil_dates.weekday >= 5).sum()
total_num_wknd = (pd.date_range(train_start, test_end).weekday >= 5).sum()

In [49]:
oil = oil.rename(columns={"dcoilwtico": "oil"})

In [50]:
# reindex oil data
oil = oil.merge(
    pd.DataFrame({"date": pd.date_range(train_start, test_end)}),
    on="date",
    how="outer",
).sort_values("date", ignore_index=True)

# fill missing values using linear interpolation
oil['oil'] = oil['oil'].interpolate(method="linear", limit_direction="both")

In [51]:
num_zero_sales = (train.groupby(["date", "store_nbr"]).sales.sum().eq(0)).sum()
total_rec = num_store * train_len
curr_rec = len(transactions.index)
missing_rec = total_rec - curr_rec - num_zero_sales

In [52]:
store_sales = train.groupby(["date", "store_nbr"]).sales.sum().reset_index()

# reindex transaction data
transactions = transactions.merge(
    store_sales,
    on=["date", "store_nbr"],
    how="outer",
).sort_values(["date", "store_nbr"], ignore_index=True)

# fill missing values with 0s for days with zero sales
transactions.loc[transactions.sales.eq(0), "transactions"] = 0.

# fill remaining missing values using linear interpolation
transactions['transaction'] = transactions.groupby("store_nbr", group_keys=False).transactions.apply(
    lambda x: x.interpolate(method="linear", limit_direction="both")
)

transactions = transactions.drop(columns=['transaction'])

In [53]:
transactions

,date,store_nbr,transactions,sales
0,2013-01-01,1,0.0,0.000000
1,2013-01-01,2,0.0,0.000000
2,2013-01-01,3,0.0,0.000000
3,2013-01-01,4,0.0,0.000000
4,2013-01-01,5,0.0,0.000000
...,...,...,...,...
91147,2017-08-15,50,2804.0,16879.121004
91148,2017-08-15,51,1573.0,20154.559000
91149,2017-08-15,52,2255.0,18600.046000
91150,2017-08-15,53,932.0,8208.189000


In [54]:
oil

,date,oil
0,2013-01-01,93.140000
1,2013-01-02,93.140000
2,2013-01-03,92.970000
3,2013-01-04,93.120000
4,2013-01-05,93.146667
...,...,...
1699,2017-08-27,46.816667
1700,2017-08-28,46.400000
1701,2017-08-29,46.460000
1702,2017-08-30,45.960000


In [55]:
merge_df = transactions.merge(
    oil,
    on = ['date'],
    how = 'left'
).sort_values(["date", "store_nbr"], ignore_index=True)

merge_df

,date,store_nbr,transactions,sales,oil
0,2013-01-01,1,0.0,0.000000,93.14
1,2013-01-01,2,0.0,0.000000,93.14
2,2013-01-01,3,0.0,0.000000,93.14
3,2013-01-01,4,0.0,0.000000,93.14
4,2013-01-01,5,0.0,0.000000,93.14
...,...,...,...,...,...
91147,2017-08-15,50,2804.0,16879.121004,47.57
91148,2017-08-15,51,1573.0,20154.559000,47.57
91149,2017-08-15,52,2255.0,18600.046000,47.57
91150,2017-08-15,53,932.0,8208.189000,47.57


In [56]:
train_df = train.merge(
    oil,
    on = ['date'],
    how = 'left'
)

test_df = test.merge(
    oil,
    on = ['date'],
    how = 'left'
)

data = pd.concat(
    [train_df, test_df], axis=0, ignore_index=True,
).sort_values(["date", "store_nbr", "family"], ignore_index=True)

data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')

In [57]:
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder

In [58]:
def apply_lgb(train_df):
    
    prediction_df = train_df[train_df['date'] >= '2017-08-16' ].copy()
    train_df = train_df[train_df['date'] < '2017-08-16' ].copy()

    # Set the index for the dataframes
    train_df.set_index(['id', 'date'], inplace=True)
    prediction_df.set_index(['id', 'date'], inplace=True)
    
        # Prepare training and prediction data
    X_train = train_df.drop(columns=['sales'])
    y_train = train_df['sales']
    X_pred = prediction_df.drop(columns=['sales'])

    # Create a LightGBM dataset for training
    train_data = lgb.Dataset(X_train, label=y_train)

    # Define and train the LightGBM model
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'num_leaves': 31,
        'learning_rate': 0.05,
    }

    num_round = 100  # Adjust the number of boosting rounds as needed
    bst = lgb.train(params, train_data, num_round)

    # Make predictions for the prediction set
    predictions = bst.predict(X_pred, num_iteration=bst.best_iteration)

    # Create a dataframe with the predictions
    prediction_df['sales'] = predictions

    train_df.reset_index(inplace=True)
    prediction_df.reset_index(inplace=True)
    
    
    
    return prediction_df


In [61]:

def label_encode_non_numeric_cols(df):
    """
    Encodes non-numeric columns in a Pandas DataFrame using LabelEncoder.
    
    Parameters:
        df (pandas.DataFrame): The DataFrame to encode.
    
    Returns:
        pandas.DataFrame: A new DataFrame with the non-numeric columns encoded.
        dict: A dictionary with original to encoded mappings for each non-numeric column.
    """
    # get the list of non-numeric column names
    non_numeric_cols = df.select_dtypes(exclude=[np.number, bool, 'category', 'datetime']).columns.tolist()
    
    # if there are no non-numeric columns, return the original DataFrame
    if not non_numeric_cols:
        return df, {}
    
    # create a new DataFrame to store the encoded columns
    encoded_df = df.copy()
    
    # dictionary to store the encodings
    encoding_dict = {}
    
    # iterate over each non-numeric column and encode its values
    for col in non_numeric_cols:
        encoder = LabelEncoder()
        encoded_df[col] = encoder.fit_transform(df[col])
        
        # store the original to encoded mappings in the dictionary
        encoding_dict[col] = dict(zip(encoder.transform(encoder.classes_), encoder.classes_ ))
    
    return encoded_df, encoding_dict

# Example usage:
# encoded_df, encodings = label_encode_non_numeric_cols(original_df)


In [62]:


data, ecoding_dict = label_encode_non_numeric_cols(data)

predictions = data.groupby('family', group_keys=False).apply(apply_lgb)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000613 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 319
[LightGBM] [Info] Number of data points in the train set: 91152, number of used features: 3
[LightGBM] [Info] Start training from score 6.086778
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000591 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 312
[LightGBM] [Info] Number of data points in the train set: 91152, number of used features: 3
[LightGBM] [Info] Start training from score 0.110266
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000601 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, y

In [ ]:
predictions = predictions.sort_values('id')

0